# CLEANING

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import warnings
warnings.simplefilter(action='ignore', category=Warning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width', 300)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_ = pd.read_json('scout_car.json', lines=True)
df  = df_.copy()

In [ ]:
df.head(5)

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0,0.01, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [ ]:
for i in df.columns:
        n=i.lower().replace(" ","_")
        df.rename(columns = {i:n}, inplace = True)
df. rename(columns = {'nr._of_doors':'nr_of_doors', 'nr._of_seats':'nr_of_seats','\ncomfort_&_convenience\n':'comfort&convenience',
                     '\nentertainment_&_media\n':'entertainment&media','\nextras\n':'extras',
                     '\nsafety_&_security\n':'safety&security',
                     'nr._of_doors':'nr_of_doors'}, inplace = True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

### 1-url

In [ ]:
df.url.value_counts().head(3)

### 2-make_model

In [ ]:
df.make_model.value_counts(dropna=False)

#####  When we examine whole data this column's information seems to be formed by "make" and "model" but let's check these columns

### 20-make , 21-model

In [ ]:
df[["make","model","make_model"]].head()

##### As it seems, make_model column has detailed information. We will add these two columns to our drop list

### 3-short_description

In [ ]:
df.short_description.value_counts(dropna=False)

In [ ]:
df[["short_description","displacement"]]

### 45-description

In [ ]:
df["description"] = [",".join(x) if type(x) == list else x for x in df["description"]]
df["description"] = df["description"].str.strip('\n').str.lstrip(',')

In [ ]:
df["description"].value_counts(dropna=False)

In [ ]:
df[["short_description","description"]]

### 4-body_type

In [ ]:
df["body_type"].value_counts(dropna=False)

##### NOTE: When we examine data, you may notice that "body" column seems to have same information. Let's find out!

### 28-body

In [ ]:
df[["body_type","body"]]

##### As we see that, They both have same information. Therefore, we will add "body" column to our drop list

### 5-price(TARGET VARIABLE)

In [ ]:
df.price.value_counts(dropna=False)

In [ ]:
df.price.describe()

##### CONCLUSION: We observed these column's dtype, missing values and values itself at the beginning. There seems to be no problem with these variable for cleaning. We will examine and visualize it detailed in our "MISSING VALUES" and "OUTLIERS" notebooks

### 6-vat

In [ ]:
df.vat.value_counts(dropna=False)

### 7-km

In [ ]:
df.km.value_counts(dropna=False)

In [ ]:
df['km'] = df['km'].str.replace(',','').str.extract('(\d+)').astype(float)

In [ ]:
df['km'].value_counts(dropna=False)

### 8 : registration

In [ ]:
df['registration'].value_counts(dropna=False)

In [ ]:
df['registration'] = df["registration"].replace("-/-", np.nan)
df['registration'] = pd.to_datetime(df['registration'])

In [ ]:
df['registration'].value_counts(dropna=False)

##### CONCLUSION: We handled dtype and unwanted character problem. We may use this column's information to create another column maybe. See the Note below.

##### NOTE: It seems that, part of this column's information is already exist in another column. Let's check this first.

### 23-first_registration

In [ ]:
df["first_registration"] = df["first_registration"].str.join("").str.strip('\n')

In [ ]:
df['first_registration'].value_counts(dropna=False)

In [ ]:
df[["registration","first_registration"]]

##### Note: This data belong to the year 2019. We will use this information to create another column

In [ ]:
df["first_registration"] = df["first_registration"].astype('float')

In [ ]:
df['age_of_car'] = 2019 - df["first_registration"]

In [ ]:
df['age_of_car']

### 9-prev_owner 

In [ ]:
df['prev_owner'].value_counts(dropna=False)

In [ ]:
df['prev_owner'] = df['prev_owner'].str.findall('\d').str.join("")

In [ ]:
df['prev_owner'].value_counts(dropna=False)

##### Note: There seems to have another column has same information. Let's check it out and compare these columns and we will decide whether drop one of them or not

### 13-previous_owners

In [ ]:
df["previous_owners"].str.join("").value_counts(dropna=False)

##### Some information is unrelated to this column obviously and might belong to another column, we will check it later with other columns. Here, we will only extract related data.

In [ ]:
df[df["previous_owners"].isnull()][["prev_owner","previous_owners"]]

##### We maybe use some values in previous owners to fill "emission class" and "co2_emission" columns

In [ ]:
df.previous_owners = [",".join(x) if type(x) == list else x for x in df.previous_owners]
df["previous_owners1"]=df.previous_owners.str.split(",").str[1].str.extract('(\d+)')[0] #co2 değerler
df["previous_owners2"]=df.previous_owners.str.split(",").str[1].str.extract('(\D+\s\d\D)')[0].str.replace('\n','') #emission class için euro değerleri
#df[df.emission_class4.isnull()]["previous_owners2"]

##### When we check NaN values in emission class and there is no index match with "emission class" column

In [ ]:
df.emission_class.isnull().sum()

In [ ]:
df["co2_emission"] = df["co2_emission"].str.join("")
df["co2_emission"] = df["co2_emission"].str.strip('\n')
df["co2_emission"] = df.co2_emission.str.extract('(\d+)')[0]

df["emission_class"] = df["emission_class"].str.join("").str.strip('\n')

In [ ]:
df["co2_emission1"]=df.co2_emission.fillna(df.previous_owners1)

In [ ]:
df["co2_emission1"].isnull().sum()

In [ ]:
df.drop(['prev_owner',"previous_owners","previous_owners1","previous_owners2"],axis=1,inplace=True)

##### We will examine co2_emission column seperately because there are other columns which consist co2_emission values. We will drop co2_emission1 afterwards.

### 10-kw

In [ ]:
df['kw'].value_counts(dropna=False)

### 11-hp

In [ ]:
df['hp'].value_counts(dropna=False)

In [ ]:
df['hp'] = df['hp'].replace('- kW',np.NaN).str.findall('\d+').str.join("")

In [ ]:
df['hp'].isnull().sum()

##### We convert unwanted character to NULL and handled problem with the data 

### 12-Type

In [ ]:
df["type"].str.join("").value_counts(dropna=False)

In [ ]:
df["type"] = df["type"].str[1]

In [ ]:
df["type"].value_counts(dropna=False)

### 37-Fuel

##### There are some values belong to "fuel" column in type variable but "fuel" column is already full. Therefore, we will extract values that we needed for type

In [ ]:
df["fuel"] = df["fuel"].str.join("").str.strip('\n')

In [ ]:
df.fuel=df.fuel.str.split("/").str[0].str.strip(' ')
benzine=["Diesel (Particulate Filter)","Diesel","Super E10 95","Super Plus 98","Super Plus E10 98",
         "Others"]
lpg=["LPG","Liquid petroleum gas","CNG","Biogas","Domestic gas H"]
def fueltype1(x):
    if x in benzine:
        return "Benzine"
    elif x in lpg:
        return "LPG/CNG"
    else:
        return "other_fuel"

In [ ]:
df.fuel=df.fuel.apply(fueltype1)

In [ ]:
df.fuel.value_counts()

### 14-next_inspection

In [ ]:
df['next_inspection'].str.join('').str.strip('\n').value_counts(dropna=False)

##### There are some values belong to other columns. We will extract them

In [ ]:
df.next_inspection = [",".join(x) if type(x) == list else x for x in df.next_inspection]
df["next_inspection2"]=df.next_inspection.str.split(",").str[1].str.extract('(\d+)')[0] #co2
df["next_inspection3"]=df.next_inspection.str.split(",").str[1].str.extract('(\D+\s\d\D)')[0].str.replace('\n','')#emission class için euro değerleri
df["next_inspection4"]=df.next_inspection.str.split(",").str[1].str.extract('(\d \D+)')[0].str.replace("\n","").str.strip(" ")#emission_label içindeki green değerleri

In [ ]:
df["co2_emission2"]=df.co2_emission1.fillna(df.next_inspection2)

In [ ]:
list_emission_label=["4 (Green)","1 (No sticker)","5 (Blue)","(Red)","3 (Yellow)"]
def emission_label_control(x): 
    if x in list_emission_label:
        return x
    else:
        return np.nan
    
df.next_inspection4=df.next_inspection4.apply(emission_label_control)
df.next_inspection4.value_counts()
df["emission_label"] = df["emission_label"].str.join('').str.strip('\n')
df["emission_label1"]=df.emission_label.fillna(df.next_inspection4)

##### We noticed that when we try to fill emission_class it doesn't effect the NULL values. Therefore, we won't fill it. We will drop emission_label1 afterwards

### 15-Inspection_new

In [ ]:
df['inspection_new'].str.join('').str.strip('\n').value_counts(dropna=False)

In [ ]:
df.inspection_new.str[0]

In [ ]:
df['Inspection'] = df['inspection_new'].str.strip('\n')
df['Inspection'] = df['Inspection'].str[:3]
df['Inspection'] = df['Inspection'].fillna('No')

In [ ]:
df['Inspection'].value_counts()

##### There are some values belong to other columns. We will extract them and fill other columns

In [ ]:
df.inspection_new = [",".join(x) if type(x) == list else x for x in df.inspection_new]
df['inspection_new1']=df.inspection_new.str.split(",").str[1].str.extract('(\d+)')[0] # co2
df['inspection_new2']=df.inspection_new.str.split(",").str[1].str.extract('(\D+\s\d\D)')[0].str.replace('\n','') # emission class için euro değerleri
df["co2_emission3"]=df.co2_emission2.fillna(df.inspection_new1)

##### We will fill emission_class below(with warranty). We willl drop unnecesarry columns afterwards

### 16-warranty

In [ ]:
df.warranty.str.join('').value_counts(dropna=False)

In [ ]:
df["warranty_"] = df.warranty.str.split(",").str[0].str.extract('(\d+)')[0].astype('float')

##### There are some values belong to other columns. We will extract them and fill other columns

In [ ]:
df.warranty = [",".join(x) if type(x) == list else x for x in df.warranty]
df['warranty2']=df.warranty.str.split(",").str[1].str.extract('(\D+\s\d\D)')[0].str.replace('\n','') # Euro 6
df["warranty3"]=df.warranty.str.split(",").str[1].str.extract('(\d+)')[0] #co2 değerleri 
df["warranty4"]=df.warranty.str.split(",").str[1].str.extract('(\d \D+)')[0].str.replace("\n","").str.strip(" ")#emission_label consist green
df['emission_class1']=df.emission_class.fillna(df.warranty2) #emission_class filled by warranty
df['emission_class2']=df.emission_class1.fillna(df.inspection_new2)
df["co2_emission4"]=df.co2_emission3.fillna(df.warranty3) #co2 emission filled by warranty3
df.warranty4=df.warranty4.apply(emission_label_control)
df["emission_label2"]=df.emission_label1.fillna(df.warranty4)

##### We will drop unnecesarry columns afterwards

### 17-full_service

In [ ]:
df['full_service'].str.join('').str.strip('\n').dropna(False)

##### There are some values belong to other columns. We will extract them and fill other columns

In [ ]:
df["full_service"]=[",".join(x) if type(x) == list else x for x in df.full_service]
df["full_service1"]=df.full_service.str.split(",").str[2].str.extract('(\D+\s\d\D)')[0].str.replace('\n','') # Euro6 
df['emission_class3']=df.emission_class2.fillna(df.full_service1) # emission_class filled by full service
df["full_service2"]=df.full_service.str.split(",").str[2].str.extract('(\d+)')[0] # co2 
df["co2_emission5"]=df.co2_emission4.fillna(df.full_service2)  # co2 emission filled by warranty3

### 38-consumption

In [ ]:
df['consumption'].str.join('').str.strip('\n').dropna(False)

In [ ]:
def parser1(x):
    if type(x) == float:
        return np.nan
    elif type(x[0]) == list:
        if x[0] != []:
            return x[0][0]
        else:
            return np.nan
    else:
        return x[1]
def parser2(x):
    if type(x) == float:
        return np.nan
    elif type(x[0]) == list:
        if x[1] != []:
            return x[1][0]
        else:
            return np.nan
    elif x[3].endswith(')'):
        return x[3]
    else:
        return np.nan
def parser3(x):
    if type(x) == float:
        return np.nan
    elif type(x[0]) == list:
        if x[2] != []:
            return x[2][0]
        else:
            return np.nan
    elif type(x[0]) != list and x[3].endswith(')'):
        return x[5]
    else:
        return np.nan


In [ ]:
df['cons_comb'] = df.consumption.apply(parser1)
df['cons_comb'] = df.cons_comb.str.extract('(\d{1,2}).\d|\d{1,3}')[0].astype('float')
df['cons_city'] = df.consumption.apply(parser2)
df['cons_city'] = df.cons_city.str.extract('(\d{1,2}.\d|\d{1,3})')[0].astype('float')
df['cons_country'] = df.consumption.apply(parser3)
df['cons_country'] = df.cons_country.str.extract('(\d{1,2}.\d)|\d{1,3}')[0].astype('float')

### 18-non-smoking_vehicle

In [ ]:
df['non-smoking_vehicle'].str.join('').str.strip('\n').value_counts(dropna=False)

In [ ]:
df['non-smoking_vehicle'] = [",".join(x) if type(x) == list else x for x in df['non-smoking_vehicle']]
df['non-smoking_vehicle1']=df['non-smoking_vehicle'].str.split(",").str[2].str.extract('(\D+\s\d\D)')[0].str.replace('\n','')#Emission_cclass için euro değerleri
df['emission_class4']=df.emission_class3.fillna(df['non-smoking_vehicle1']) #emission_class non-smoking ile dolduruldu
df['non-smoking_vehicle2']=df['non-smoking_vehicle'].str.split(",").str[2].str.extract('(\d+)')[0] #co2 değerleri
df["co2_emission6"]=df.co2_emission5.fillna(df['non-smoking_vehicle2']).astype(float) #co2 emission non-smoking_vehicle2 ile dolduruldu
def value_check(x):
    if x<50:
        return np.nan
    else:
        return x
#bu fonksiyon ile co2_emission6 da biriken 2 4 6 gibi başka yerlerden gelen anlamsız değerler temizlendi 
df.co2_emission6=df.co2_emission6.apply(value_check)

In [ ]:
df['non-smoking_vehicle3']=df['non-smoking_vehicle'].str.split(",").str[2].str.extract('(\d \D+)')[0].str.replace("\n","").str.strip(" ")#emission_label içindeki green değerleri
df['non-smoking_vehicle3']=df['non-smoking_vehicle3'].apply(emission_label_control)
df["full_service3"]=df.full_service.str.split(",").str[2].str.extract('(\d \D+)')[0].str.replace("\n","").str.strip(" ")#emission_label içindeki green değerleri
df.full_service3=df.full_service3.apply(emission_label_control)
df["emission_label3"]=df.emission_label2.fillna(df.full_service3) 
df["emission_label4"]=df.emission_label3.fillna(df['non-smoking_vehicle3']) #null değerler azaldı

### 39-co2_emission

In [ ]:
df["co2_emission"].str.join("").str.strip('\n').value_counts(dropna=False)

### 40-emission_class

In [ ]:
df["emission_class"].str.join("").str.strip('\n').value_counts(dropna=False)

In [ ]:
df.emission_class4.value_counts(dropna=False)

### 46-emission_label

In [ ]:
df["emission_label"].value_counts(dropna=False)

In [ ]:
df.emission_label4.value_counts(dropna=False)

In [ ]:
def emission_label_control(x): 
    if x in list_emission_label:
        return x
    else:
        return np.nan
    
df.next_inspection4=df.next_inspection4.apply(emission_label_control)
df.next_inspection4.value_counts()
df["emission_label"] = df["emission_label"].str.join('').str.strip('\n')
df["emission_label1"]=df.emission_label.fillna(df.next_inspection4)

### 19-null

In [ ]:
df["null"] = df["null"].str.join("")
df["null"].value_counts(dropna=False)

##### We will add this column to our drop list. All values are same and have no information

### 22-offer_number

In [ ]:
df["offer_number"] = df["offer_number"].str.join("").str.strip('\n')
df["offer_number"].value_counts(dropna=False)

### 24-body_color

In [ ]:
df["body_color"] = df["body_color"].str.join("").str.strip('\n')
df.body_color.value_counts(dropna=False)

In [ ]:
df.groupby("body_color")['price'].mean()

### 25-paint_type

In [ ]:
df["paint_type"] = df["paint_type"].str.join("").str.strip('\n')
df.paint_type.value_counts(dropna=False)

In [ ]:
df.groupby("paint_type")['price'].mean()

### 26-body_color_original

In [ ]:
df["body_color_original"] = df["body_color_original"].str.join("").str.strip('\n')
df["body_color_original"].value_counts(dropna=False)

### 27-upholstery

In [ ]:
df["upholstery"].str.join('').str.strip('\n').value_counts(dropna=False)

In [ ]:
df.upholstery = [item[0] if type(item) == list else item for item in df.upholstery]
df.upholstery = df.upholstery.str.strip('\n').str.split(', ')
u_type  = ['Cloth', 'Part leather', 'Full leather', 'Velour', 'alcantara']
df['Upholstery_Type'] = df.upholstery.apply(lambda x : x[0] if type(x) == list and x[0] in u_type else np.nan)

In [ ]:
df.upholstery.str.join('').value_counts(dropna = False)

In [ ]:
color = ['Black', 'Grey', 'Brown', 'Beige', 'White', 'Blue', 'Red', 'Yellow', 'Orange']

def finder(x):
    if type(x) == list and len(x) == 2 :
        return x[1]
    elif type(x) == list and x[0] in color :
        return x[0]
    else :
        return np.nan
df['Upholstery_color'] = df.upholstery.apply(finder)

In [ ]:
df.drop('upholstery', axis = 1, inplace = True)

### 29-nr_of_doors

In [ ]:
df["nr_of_doors"] = df["nr_of_doors"].str.join("").str.strip('\n')
df["nr_of_doors"].value_counts(dropna=False)

In [ ]:
df.groupby("nr_of_doors")['price'].mean()

### 30-nr_of_seats

In [ ]:
df["nr_of_seats"] = df["nr_of_seats"].str.join("").str.strip('\n')
df["nr_of_seats"].value_counts(dropna=False)

In [ ]:
df.groupby("nr_of_seats")['price'].mean()

### 31-model_code

In [ ]:
df["model_code"] = df["model_code"].str.join("").str.strip('\n')
df["model_code"].value_counts(dropna=False)

### 32-gearing_type

In [ ]:
df["gearing_type"] = df["gearing_type"].str.join("").str.strip('\n')
df["gearing_type"].value_counts(dropna=False)

In [ ]:
df.groupby("gearing_type")['price'].mean()

In [ ]:
df["gearing_type"].replace('Semi-automatic','Automatic',inplace=True)

In [ ]:
df.groupby("gearing_type")['price'].mean()

### 33-displacement

In [ ]:
df["displacement"] = df["displacement"].str.join("").str.strip('\n').str.findall('\d+').str.join("").str.strip(',').astype('float')
df["displacement"].value_counts(dropna=False)

### 34-cylinders

In [ ]:
df["cylinders"] = df["cylinders"].str[0].str.strip('\n')
df["cylinders"].value_counts(dropna=False)

### 35-weight

In [ ]:
df["weight"] = df["weight"].str.join("").str.strip('\n').str.findall('\d+').str.join("").str.strip(',').astype('float')
df["weight"].value_counts(dropna=False)

In [ ]:
df.groupby("weight")['price'].mean()

##### It seems that there is no correlation between the weight of the car and price. We will add this column to our drop list

### 36-drive_chain

In [ ]:
df["drive_chain"] = df["drive_chain"].str.join("").str.strip('\n')
df["drive_chain"].value_counts()

### 41-comfort&convenience

In [ ]:
df["comfort&convenience"].str.join('').value_counts(dropna=False).head(10)

In [ ]:
df["comfort&convenience"] = [','.join(item) if type(item) == list else item for item in  df['comfort&convenience']]

### 42-entertainment&media

In [ ]:
df["entertainment&media"].str.join('').value_counts(dropna=False).head(5)

In [ ]:
df["entertainment&media"].str.join("").nunique()

### 43- extras

In [ ]:
df["extras"].str.join('').value_counts(dropna=False).head(5)

In [ ]:
df["extras"].str.join("").nunique()

### 44-safety&security

In [ ]:
df["safety&security"].str.join('').value_counts(dropna=False).head(5)

In [ ]:
df["safety&security"].str.join("").nunique()

### 47-gears

In [ ]:
df["gears"] = [",".join(x) if type(x) == list else x for x in df["gears"]]
df["gears"] = df["gears"].str.strip('\n')

In [ ]:
df["gears"].value_counts(dropna=False)

In [ ]:
df["gears"] = df["gears"].astype(str)

In [ ]:
df.groupby('gears')['price'].mean()

### 48-country_version

In [ ]:
df["country_version"] = [",".join(x) if type(x) == list else x for x in df["country_version"]]
df["country_version"] = df["country_version"].str.strip('\n')

In [ ]:
df.country_version.value_counts(dropna=False)

### 49-Electricity consumption, 50-Last Service Date, 51-other_fuel_types, 52-availability, 53-last_timing_belt_service_date, 54-Available from

##### These columns consists more than %90 NULL values and we can't fill them by checking other columns. We will add these columns to our drop list

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
drop_list = ["url","make","model","short_description","description","body","registration","full_service", "first_registration","kw","next_inspection","non-smoking_vehicle","emission_label","null","offer_number","body_color","paint_type","body_color_original","nr_of_seats","nr_of_doors","model_code","cylinders","weight","drive_chain","country_version","electricity_consumption","last_service_date","other_fuel_types","availability","last_timing_belt_service_date", "available_from","Upholstery_color","emission_label4","emission_label3","full_service3","non-smoking_vehicle3","non-smoking_vehicle2","emission_class","emission_class4","non-smoking_vehicle1","co2_emission5","full_service2","emission_class3","full_service1","emission_label2","co2_emission4","emission_class2","emission_class1","warranty4","warranty3","warranty2","co2_emission3","inspection_new2","inspection_new1","emission_label1","co2_emission2","next_inspection4","next_inspection3","next_inspection2","co2_emission1","consumption","inspection_new","warranty"]

In [ ]:
df.drop(drop_list ,axis=1,inplace=True)

In [ ]:
df.head(2)

In [ ]:
df["Inspection"].value_counts()

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df.to_csv('Capstone_Cleaning.csv')